### Importação das bibliotecas necessárias para a raspagem de dados.

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import requests
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.

### Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [2]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9298334
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9298333
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9298332
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9298331
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9298327
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9298325
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298323
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298322
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298321
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298320
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298319
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298318
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298317
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9298316
FINISHED

In [4]:
#list_finished_routes
#list_pending_routes

[['https://www.openstreetmap.org/user/sunnypilot/traces/9282758'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282757'],
 ['https://www.openstreetmap.org/user/dragonpilot/traces/9282755'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282754'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282753'],
 ['https://www.openstreetmap.org/user/dragonpilot/traces/9282752'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282751'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282750'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282749'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282748'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282747'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282746'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282745'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282744'],
 ['https://www.openstreetmap.org/user/sunnypilot/traces/9282

### Acessando a telas para efetuar os downloads.

In [3]:
## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
navegador = webdriver.Chrome(service=servico)
for list_route in list_routes:
    sleep(5)
    url = list_route[0]
    navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
    navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
navegador.close()

### Executando de uma só vez

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import requests
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())

options = Options()
options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
#options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.

# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
#navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)
for list_route in list_routes:
    sleep(5)
    url = list_route[0]
    navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
    navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
navegador.close()

FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9300222
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9300221
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9300220
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9300219
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9300218
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9300217
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9300216
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9300215
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9300214
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9300213
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9300212
FINISHED: https://www.openstreetmap.org/user/sunnypilot/traces/9300211
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9300208
FINISHED: https://www.openstreetmap.org/user/dragonpilot/traces/9300207
FINIS

### Movendo os arquivos da pasta Download para o DATALAKE

In [2]:
import os
import shutil
DATALAKE_STAGE = 'DATALAKE_STAGE'  # Substitua pelo caminho do diretório de destino
DOWNLOADS = '/home/thiago/Downloads/'

In [3]:
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

In [4]:
files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
for name_files in files_to_move:
    origin_path = os.path.join(DOWNLOADS, name_files)
    destiny_path = os.path.join(DATALAKE_STAGE, name_files)
    os.rename(origin_path, destiny_path)
    print(f"Arquivo {name_files} movido para {destiny_path}")

Arquivo 9305211.gpx movido para DATALAKE_STAGE/9305211.gpx
Arquivo 9305227.gpx movido para DATALAKE_STAGE/9305227.gpx
Arquivo 9305223.gpx movido para DATALAKE_STAGE/9305223.gpx
Arquivo 9305216.gpx movido para DATALAKE_STAGE/9305216.gpx
Arquivo 9305225.gpx movido para DATALAKE_STAGE/9305225.gpx
Arquivo 9305205.gpx movido para DATALAKE_STAGE/9305205.gpx
Arquivo 9305221.gpx movido para DATALAKE_STAGE/9305221.gpx
Arquivo 9305229.gpx movido para DATALAKE_STAGE/9305229.gpx
Arquivo 9305218.gpx movido para DATALAKE_STAGE/9305218.gpx
Arquivo 9305219.gpx movido para DATALAKE_STAGE/9305219.gpx
Arquivo 9305208.gpx movido para DATALAKE_STAGE/9305208.gpx
Arquivo 9305207.gpx movido para DATALAKE_STAGE/9305207.gpx
Arquivo 9305226.gpx movido para DATALAKE_STAGE/9305226.gpx
Arquivo 9305220.gpx movido para DATALAKE_STAGE/9305220.gpx
Arquivo 9305228.gpx movido para DATALAKE_STAGE/9305228.gpx
Arquivo 9305209.gpx movido para DATALAKE_STAGE/9305209.gpx
Arquivo 9305217.gpx movido para DATALAKE_STAGE/9305217.g

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP##
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()